In [15]:
%load_ext autoreload
%autoreload 2
import sys
import torch
from torch import nn
sys.path.append('..')
# sys.path.append('/system/user/beck/pwbeck/projects/regularization/ml_utilities')
from pathlib import Path
from typing import Union
from ml_utilities.torch_models.base_model import BaseModel
from ml_utilities.torch_models.fc import FC
from ml_utilities.torch_models import get_model_class
from omegaconf import OmegaConf

from erank.utils import load_directions_matrix_from_task_sweep
import matplotlib.pyplot as plt
from erank.output_loader.job_output import JobResult
from erank.utils import get_best_model_idx

gpu_id = 0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## JobResult class

In [16]:
job_dir = '/system/user/beck/pwbeck/projects/regularization/erank/outputs/mnist-11.7.0-lenet_rotatedtasks--221015_124528/outputs/mnist-11.7.0-lenet_rotatedtasks--init_model_step-0-rotation_angle-0.0-seed-0--221015_195056'

In [17]:
jobr = JobResult(job_dir = job_dir)

In [18]:
str(jobr)

'/system/user/beck/pwbeck/projects/regularization/erank/outputs/mnist-11.7.0-lenet_rotatedtasks--221015_124528/outputs/mnist-11.7.0-lenet_rotatedtasks--init_model_step-0-rotation_angle-0.0-seed-0--221015_195056'

In [19]:
jobr.is_valid_job()

True

In [20]:
jobr.get_config()

{'run_config': {'exec_type': 'parallel', 'hostname': 'node01', 'gpu_ids': [0, 1, 2], 'runs_per_gpu': 6, 'wandb': {'init': {'tags': ['${config.experiment_data.experiment_tag}_exps', 'run_handler'], 'notes': None, 'group': '${config.experiment_data.experiment_tag}', 'job_type': 'run_handler'}}}, 'seeds': [0], 'sweep': {'type': 'grid', 'axes': [{'parameter': 'trainer.init_model_step', 'vals': [0, 5, 15, 50, 100]}, {'parameter': 'data.dataset_kwargs.rotation_angle', 'vals': 'linspace(0,180,360,endpoint=True)'}]}, 'start_num': 0, 'config': {'experiment_data': {'entity': 'jkuiml-fsl', 'project_name': 'sparsity', 'experiment_tag': '11.7', 'experiment_type': 'startnum_${start_num}', 'experiment_name': 'mnist-11.7.0-lenet_rotatedtasks--init_model_step-0-rotation_angle-0.0-seed-0', 'experiment_dir': None, 'experiment_notes': 'Hyperparameter search.', 'job_name': None, 'seed': 0, 'hostname': 'node01', 'gpu_id': 1}, 'model': {'name': 'fc', 'model_kwargs': {'input_size': 784, 'hidden_sizes': [300, 

In [26]:
jobr.job_dir.stem

'mnist-11.7.0-lenet_rotatedtasks--init_model_step-0-rotation_angle-0'

In [29]:
jobr.get_summary()

,best_step,best_val_score,trainer.init_model_step,data.dataset_kwargs.rotation_angle
mnist-11.7.0-lenet_rotatedtasks--init_model_step-0-rotation_angle-0,1550,0.97675,0,0.0


In [38]:
jobr.get_data_log('train')

,epoch,train_step,loss_CrossEntropyLoss,Accuracy,time_last_train_epoch_in_s,log_step
0,1,375,0.299773,0.911083,0.000000,383
1,2,750,0.114342,0.964688,43.808768,767
2,3,1125,0.074017,0.977687,53.615765,1150
3,4,1500,0.053052,0.982938,37.999984,1534
4,5,1875,0.037440,0.987938,48.987577,1917
5,6,2000,0.029647,0.991000,48.566147,2046
